In [3]:
import tweepy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk.sentiment.vader import SentimentIntensityAnalyzer

plt.style.use('fivethirtyeight')



In [4]:
consumer_key = ""
consumer_secret = ""
access_token = ""
access_token_secret = ""

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

user = api.me()
print(user.name)



TweepError: Unable to get username, invalid oauth token!

In [ ]:
weets = []

for page in range(1,5):
    tweets.extend(api.user_timeline(screen_name="realDonaldTrump", count=200, page=page))

print("Number of tweets extracted: {}. \n".format(len(tweets)))


In [ ]:
for tweet in tweets[:5]:
    print(tweet.text)

In [ ]:
own_tweets = [tweet for tweet in tweets if tweet.retweeted == False and "RT @" not in tweet.text]

df = pd.DataFrame(data=[[tweet.created_at, tweet.text, len(tweet.text), tweet.id, tweet.favorite_count, tweet.retweet_count] for tweet in own_tweets], columns=['Date', 'Tweet', 'Length', 'ID', 'Likes', 'Retweets'])

In [ ]:
df.head()

In [ ]:
df[df['Likes']==df['Likes'].max()]

In [ ]:
df['Likes'].describe()

In [ ]:
df['Likes'].plot(kind='hist', bins=20, figsize=(5,5), ec='black')
plt.xlabel('Likes Count')
plt.ylabel('Frequency')
plt.title('# of Likes by Tweet')
plt.show()

In [ ]:
vader = SentimentIntensityAnalyzer()
f = lambda tweet: vader.polarity_scores(tweet)['compound']
df['Sentiment'] = df['Tweet'].apply(f)
df['Date'] = pd.to_datetime(df['Date']).dt.date
df.head()

In [ ]:
df['Sentiment'].plot(kind='hist', bins=20, figsize=(5,5), ec='black')
plt.xlabel('Sentiment')
plt.ylabel('Frequency')
plt.title('Sentiment of Tweets by Trump')
plt.show()

In [ ]:
date_df = df.groupby(['Date']).mean().reset_index()

In [ ]:
date_df.plot(kind='line', x='Date', y='Sentiment', figsize=(5,5), ylim=[-1,1])
plt.axhline(y=0, color='black')
plt.ylabel('Average Sentiment')
plt.title('Daily Average Sentiment of Tweets')

In [ ]:
from wordcloud import WordCloud, STOPWORDS

text = " ".join(text for text in df.Tweet)

stopwords = set(STOPWORDS)
stopwords.update(["HTTPS", "CO"])

wordcloud = WordCloud(stopwords=stopwords, background_color="white").generate(text)

plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()